0 - create dataset  
1 - run experiment  
2 - plot counterfactuals  
3 - plot embeddings  
4 - plot results  

In [1]:
import pickle

from src.evaluation.evaluator_manager import EvaluatorManager
from src.evaluation.evaluator_manager_do import EvaluatorManager as PairedEvaluatorManager
from src.evaluation.evaluator_manager_triplets import EvaluatorManager as TripletsEvaluatorManager

from src.utils.context import Context

from utils_martina.my_utils import *

In [2]:
# Set patient and record (tutti con gli stessi elettrodi)

# observations = ["chb01_03", "chb01_04", "chb01_16"]

# observations = ["chb01_03", "chb01_04", "chb01_15", "chb01_16", "chb01_18", "chb01_21", "chb01_26"]
# observations = ["chb02_16", "chb02_19"]
observations = ["chb03_01", "chb03_02", "chb03_03", "chb03_04", "chb03_34", "chb03_35", "chb03_36"]
# observations = ["chb04_28", "chb04_08"]
# observations = ["chb05_06", "chb05_13", "chb05_16", "chb05_17", "chb05_22"]
# observations = ["chb06_01", "chb06_04"] #, "chb06_09", "chb06_10", "chb06_13", "chb06_18", "chb06_24"]

create_dataset_json(observations)

In [3]:
logs_path = "..\\..\\explainability\GRETEL-repo\\output\\logs\\"
eval_manager_path = "..\\..\\explainability\GRETEL-repo\\output\\eval_manager\\"

## Removing cache dataset

In [4]:
remove_cache_dataset = True

if remove_cache_dataset:
    folder = ".\data\cache\datasets"

    for file in os.listdir(folder):
        if file.startswith("GCS-"):  # Checks if the file begins with "GCS-"
            cache_path = os.path.join(folder, file)
            if os.path.isfile(cache_path):
                os.remove(cache_path)
                print(f"Removed: {cache_path}")

Removed: .\data\cache\datasets\GCS-7461d1da0d4c1403016b336840abe8b6


## Run Gretel experiment

In [5]:
# NOTA: Se si blocca, andare a vedere se ci sono dei file OracleTorch provenienti dal futuro

In [ ]:
path = 'config/GCS-GCN.jsonc'
# path = 'config/GCS-SVM.jsonc'
# path = 'config/GCS-KNN.jsonc'

print(observations)

print(f"Generating context for: {path}")
context = Context.get_context(path)
context.run_number = -1

context.logger.info(f"Executing: {context.config_file} Run: {context.run_number}")
context.logger.info(
    "Creating the evaluation manager......................................................."
)

if 'doe-triplets' in context.conf:
    context.logger.info("Creating the TRIPLET evaluators........................................................")
    eval_manager = TripletsEvaluatorManager(context)
if 'do-pairs' in context.conf:
    context.logger.info("Creating the PAIRED evaluators...............................................................") ### Usiamo questo
    eval_manager = PairedEvaluatorManager(context)
else:
    context.logger.info("Creating the evaluators...............................................................")
    eval_manager = EvaluatorManager(context)

context.logger.info(
    "Evaluating the explainers............................................................."
)

eval_manager.evaluate()

# Additional stuff
# context.logger.info(f"graph_ids: {eval_manager._evaluators[0].get_instance_and_counterfactual_ids()}") # NON È STATO SISTEMATO PER CONTEMPLARE IL CASO DI PIU' PAZIENTI
context.logger.info(f"graph_metrics: {eval_manager._evaluators[0].get_instance_and_counterfactual_graph_metrics()}")

['chb03_01', 'chb03_02', 'chb03_03', 'chb03_04', 'chb03_34', 'chb03_35', 'chb03_36']
Generating context for: config/GCS-GCN.jsonc
2025-03-11 18:28:50,081 | INFO | 10400 - Executing: config/GCS-GCN.jsonc Run: -1
2025-03-11 18:28:50,123 | INFO | 10400 - Creating the evaluation manager.......................................................
2025-03-11 18:28:50,161 | INFO | 10400 - Creating the PAIRED evaluators...............................................................
2025-03-11 18:28:50,289 | INFO | 10400 - Creating: GCS-7461d1da0d4c1403016b336840abe8b6
2025-03-11 18:28:50,289 | INFO | 10400 - Instantiating: src.dataset.generators.gcs.GCS
2025-03-11 18:29:08,136 | INFO | 10400 - Saved: GCS-7461d1da0d4c1403016b336840abe8b6
2025-03-11 18:29:08,234 | INFO | 10400 - Created: GCS-7461d1da0d4c1403016b336840abe8b6
2025-03-11 18:29:08,340 | INFO | 10400 - Instantiating: src.oracle.nn.gcn.DownstreamGCN
2025-03-11 18:29:08,349 | INFO | 10400 - Instantiating: torch.optim.RMSprop
2025-03-11 18:2

In [ ]:
# Save eval_manager to file
file_name = get_most_recent_file(logs_path).split('.')[0]

with open(eval_manager_path + file_name + ".pkl", 'wb') as f:
    pickle.dump(eval_manager, f)

## _______________________________________________________________

In [ ]:
import datetime

now = datetime.datetime.now()
print("Ultima esecuzione completa:", now.strftime("%d/%m/%Y, ore %H:%M"))

Ultima esecuzione completa: 11/03/2025, ore 13:49


In [ ]:
import winsound

winsound.Beep(440, 500)
winsound.Beep(400, 500)
winsound.Beep(440, 500)